This is a Markdown cell

In [ ]:
import numpy as np
import pyshtools
import shtaper
import copy

In [ ]:
# to do

In [ ]:
grid_step = 0.25
SH_lmax_grid = int(90/grid_step - 1)  # max degree to get a grid_step spaced grid
obs_height = 10e3  # m
obs_a = pyshtools.constant.a_wgs84.value + obs_height

In [ ]:
GGM_cilm, GGM_gm, GGM_r0 = \
    pyshtools.shio.read_icgem_gfc('XGM2019.gfc', lmax=SH_lmax_grid)
GTM_cilm, GTM_gm, GTM_r0 = \
    pyshtools.shio.read_icgem_gfc('dV_ELL_Earth2014.gfc', lmax=SH_lmax_grid)

In [ ]:
GGM_coeffs = pyshtools.SHGravCoeffs.from_array(
    GGM_cilm, GGM_gm, GGM_r0,
    omega=pyshtools.constant.omega_wgs84.value,
    lmax=SH_lmax_grid)
GTM_coeffs = pyshtools.SHGravCoeffs.from_array(
    GTM_cilm, GTM_gm, GTM_r0,
    omega=0,
    lmax=SH_lmax_grid)

In [ ]:
if not(GGM_r0 == GTM_r0):
    raise AssertionError("gravity model and terrain effect model should share the same r0")
if not(GGM_gm == GTM_gm):
    raise AssertionError("gravity model and terrain effect model should share the same gm")

In [ ]:
LP_weights = \
    shtaper.taper_weights(l_start=300, l_stop=SH_lmax_grid, taper="gentle")

GGM_coeffs_LP = copy.deepcopy(GGM_coeffs)
GGM_coeffs_LP.coeffs = np.multiply(GGM_coeffs_LP.coeffs, LP_weights)

GTM_coeffs_LP = copy.deepcopy(GTM_coeffs)
GTM_coeffs_LP.coeffs = np.multiply(GTM_coeffs_LP.coeffs, LP_weights)

In [ ]:
GOCO06s_grid = pyshtools.SHGravCoeffs.expand(
    GGM_coeffs_LP,
    a=obs_a, f=pyshtools.constant.f_wgs84.value,
    extend=False, lmax=SH_lmax_grid, normal_gravity=True)
GOCO06s_grid_gd = pyshtools.SHGrid.from_array(GOCO06s_grid.total.data*1e5)
E2014_grid = pyshtools.SHGravCoeffs.expand(
    GTM_coeffs_LP,
    a=obs_a, f=pyshtools.constant.f_wgs84.value,
    extend=False, lmax=SH_lmax_grid, normal_gravity=False)
E2014_grid_rad = pyshtools.SHGrid.from_array(E2014_grid.rad.data * -1e5)  # to mGal and positive downwards

# %% apply terrain correction
# perform difference of potential (adimensional) grid, then expand to SH coefficients, then to gravity grids
# normal gravity is removed after applying the reduction
# (potential issue: what if gm and r0 are different between GGM and terrain effect model?)

GGM_PotGrid_adim = (pyshtools.SHCoeffs.from_array(GGM_cilm)).expand()
GTM_PotGrid_adim_low = (pyshtools.SHCoeffs.from_array(GTM_cilm)).expand()
Vbg_Coeffs = (
    pyshtools.SHGrid.from_array(
        GGM_PotGrid_adim.data - GTM_PotGrid_adim_low.data
    )).expand()
Vbg_GravCoeffs = pyshtools.SHGravCoeffs.from_array(Vbg_Coeffs.coeffs,
                                                   gm=GGM_gm, r0=GGM_r0,
                                                   omega=pyshtools.constant.omega_wgs84.value)
Vbg_GravGrid = Vbg_GravCoeffs.expand(normal_gravity=True,
                                     a=obs_a, f=pyshtools.constant.f_wgs84.value)
Vbg_GravGrid_total = pyshtools.SHGrid.from_array(Vbg_GravGrid.total.data * 1e5)